In [ ]:
!git clone https://github.com/Subhadip7/yolov8-multiple-vehicle-detection.git


In [ ]:
%cd yolov8-multiple-vehicle-detection


In [ ]:
!pip install -r requirements.txt


In [ ]:
!ls yolov8-multiple-vehicle-detection


In [ ]:
!cat yolov8-multiple-vehicle-detection/mainh.py


In [ ]:
!pip install ultralytics opencv-python-headless cvzone

In [ ]:
%%writefile yolov8-multiple-vehicle-detection/mainh.py
import cv2
import pandas as pd
from ultralytics import YOLO
import cvzone
from tracker import *

# Load YOLOv8 model
model = YOLO('yolov8s.pt')

# Open video
cap = cv2.VideoCapture('/kaggle/working/yolov8-multiple-vehicle-detection/yolov8-multiple-vehicle-detection/tf.mp4')

# Load COCO class names
with open("/kaggle/working/yolov8-multiple-vehicle-detection/yolov8-multiple-vehicle-detection/coco.txt", "r") as my_file:
    class_list = my_file.read().split("\n")

# Video properties
fps = cap.get(cv2.CAP_PROP_FPS)
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter('output.mp4', fourcc, fps, (1020, 500))

# Initialize counters and tracker
count = 0
car_count = 0
bus_count = 0
truck_count = 0
tracker = Tracker()
cy1 = 184
offset = 8

while True:
    ret, frame = cap.read()
    if not ret:
        break

    count += 1
    if count % 3 != 0:
        continue

    frame = cv2.resize(frame, (1020, 500))

    # Predict
    results = model.predict(frame)
    detections = results[0].boxes.data
    px = pd.DataFrame(detections.cpu()).astype("float")  # <-- fixed .cpu()

    cars, buses, trucks = [], [], []

    for index, row in px.iterrows():
        x1 = int(row[0])
        y1 = int(row[1])
        x2 = int(row[2])
        y2 = int(row[3])
        d = int(row[5])
        c = class_list[d]
        if 'car' in c:
            cars.append([x1, y1, x2, y2])
        elif 'bus' in c:
            buses.append([x1, y1, x2, y2])
        elif 'truck' in c:
            trucks.append([x1, y1, x2, y2])

    cars_boxes = tracker.update(cars)
    buses_boxes = tracker.update(buses)
    trucks_boxes = tracker.update(trucks)

    cv2.line(frame, (1, cy1), (1018, cy1), (0, 255, 0), 2)

    # Count vehicles
    for bbox in cars_boxes:
        cx = int((bbox[0] + bbox[2]) / 2)
        cy = int((bbox[1] + bbox[3]) / 2)
        if cy1 - offset < cy < cy1 + offset:
            car_count += 1

    for bbox in buses_boxes:
        cx = int((bbox[0] + bbox[2]) / 2)
        cy = int((bbox[1] + bbox[3]) / 2)
        if cy1 - offset < cy < cy1 + offset:
            bus_count += 1

    for bbox in trucks_boxes:
        cx = int((bbox[0] + bbox[2]) / 2)
        cy = int((bbox[1] + bbox[3]) / 2)
        if cy1 - offset < cy < cy1 + offset:
            truck_count += 1

    # Draw boxes
    for bbox in cars_boxes + buses_boxes + trucks_boxes:
        cv2.rectangle(frame, (bbox[0], bbox[1]), (bbox[2], bbox[3]), (255, 0, 255), 2)
        cvzone.putTextRect(frame, f'ID: {bbox[4]}', (bbox[0], bbox[1]), 1, 1)

    # Display counts
    cv2.rectangle(frame, (10, 10), (300, 100), (0, 0, 0), -1)
    cv2.putText(frame, f"Cars: {car_count}", (20, 40), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    cv2.putText(frame, f"Buses: {bus_count}", (20, 70), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 0), 2)
    cv2.putText(frame, f"Trucks: {truck_count}", (20, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2)

    # Write frame to output video
    out.write(frame)

print(f'Total car count: {car_count}')
print(f'Total bus count: {bus_count}')
print(f'Total truck count: {truck_count}')

cap.release()
out.release()


In [ ]:
!python yolov8-multiple-vehicle-detection/mainh.py


In [ ]:
from IPython.display import HTML
from base64 import b64encode

mp4 = open('output.mp4','rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width=600 controls>
    <source src="%s" type="video/mp4">
</video>
""" % data_url)
